<a href="https://colab.research.google.com/github/TeaWithLucas/MLBirds/blob/main/implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import cv2
import numpy as np
import re
import seaborn as sns
import operator
from IPython import get_ipython
ipy = get_ipython()
if ipy is not None:
    ipy.run_line_magic('matplotlib', 'inline')

In [ ]:
git_fldr = '/content/MLBirds/'

if os.path.exists(git_fldr):
  %cd '/content/MLBirds/'
  !git pull
else:
  !git clone https://github.com/TeaWithLucas/MLBirds.git
  %cd '/content/MLBirds/'

import data_load as data